# **AI Storytelling**

<img src="assets/logo.jpg" width="500" height="300">

This is Natural Language Processing platform that targets on conversion of short stories to audiobooks with features:
- characters extraction from text,
- voice generation by dialogues, narration and characters,
- musical background creation by text,
- activities sound generation by lines, and
- combination of all above features to create the audiobook.

Based on the success of the project, it could be extended with image or video processing features in upcoming days.

### Import Libraries and Functions

In [33]:
# basic libraries
import re
import os
import pandas as pd

In [5]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# nlp libraries
import spacy
nlp = spacy.load("en_core_web_lg")

In [7]:
# other libraries
import joblib

### Files and Dataframes Defining

In [306]:
# checking stories
os.listdir("stories")

['beautiful garden.txt',
 'cafe at midnight.txt',
 'the painted door.txt',
 'threads of imagination.txt']

In [307]:
# input title
title = input("Enter text file name:")

Enter text file name: threads of imagination


In [308]:
# opening text file
story = f"stories/{title}.txt"
with open(story, "r") as f:
    text = f.read()

In [309]:
# create dataframes
df_characters = pd.DataFrame(columns=["cid", "name", "frequency", "features"])
df_lines = pd.DataFrame(columns=["pid", "ndid", "character", "dialogue", "narration"])

# 1. Lines Identification

In [310]:
# converting text to paragraphs
paragraphs = text.split("\n")
non_empty_paragraphs = list(filter(lambda x: x != '', paragraphs))

In [311]:
# function to identify narrations and dialogues
def identify_narrations_and_dialogues(paragraph):
    """
    :param paragraph: string of paragraph in a story
    :return: list of tuples in (id, name_of_speaker, dialogue, narration) format
    """
    divisions = paragraph.split('"')
    divisions = list(filter(lambda x: x != '', divisions))
    i, spoken = 0, False
    narrations_and_dialogues = []
    for division in divisions:
        start_index = paragraph.find(division)
        end_index = start_index + len(division) - 1
        try:
            if '"'==paragraph[start_index-1:start_index] and '"'==paragraph[end_index+1:end_index+2] and spoken==False:
                # dialogues
                narrations_and_dialogues.append((i, None, division, None))
                spoken = True
            else:
                # narrations
                narrations_and_dialogues.append((i, None, None, division))
                spoken = False
        except:
            # narrations
            narrations_and_dialogues.append((i, None, None, division))
            spoken = False
        i += 1
    return narrations_and_dialogues


In [312]:
# identifying lines (narrations or dialogues) from each paragraphs
pid_num = 0
for paragraph in non_empty_paragraphs:
    for row in [(pid_num,)+nad for nad in identify_narrations_and_dialogues(paragraph)]:
        df_lines = df_lines._append(pd.Series(row, index=df_lines.columns), ignore_index=True)
    pid_num += 1

In [313]:
# set index using paragraph and nd
df_lines.set_index(['pid', 'ndid'], inplace=True)

In [314]:
# narrations and dialogues
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1         None                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1         None                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1         None                 What's the story behind this book?   
3   0         None                                               None   
    1         None  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Deep in the heart of a sprawling city, where s...  
    1                                                  None  
    2      was a haven for those seeking refuge from the...  
1   0     One rainy afternoon, as the sound of raindrops...  
    1                                                  None  
2   0                  Intrigued, Oliver approached Clara.   
    1                                                  None  
3   0                Clara, a twinkle in her eye, replied,   
    1                                                  None  
4   0     Oliver, fueled by curiosity, decided to purcha...

# 2. Characters Identification

In [315]:
# function to find subjects speaking corresponding dialogues
def identify_subjects(sentence):
    """
    :param sentence: string of sentence in the story
    :return: individual speaker
    """
    tokens = []
    individual, individuals = '', []
    
    for tok in nlp(sentence):
        tokens.append([tok, tok.dep_])
        
    for i in range(len(tokens)-1, -1, -1):
        if tokens[i][1] == "nsubj":
            individual = str(tokens[i][0])
        elif (tokens[i][1] in ['comp'] and individual != ''):
            individual = str(tokens[i][0]) + " " + individual
        else:
            individuals.append(individual)
            
    individuals.append(individual)
    return list(set([ind for ind in individuals if ind != '']))

In [316]:
# finding semantic similarity
def calculating_semantic_similarity(subject, sentence):
    """
    :param subject: list of strings available in a sentence
    :return: similarity score
    """
    subject_doc = nlp(subject)
    sentence_doc = nlp(sentence)
    
    subject_vec = subject_doc.vector
    sentence_vec = sentence_doc.vector
    
    subject_vec_norm = subject_doc.vector_norm
    sentence_vec_norm = sentence_doc.vector_norm
    
    if subject_vec_norm > 0 and sentence_vec_norm > 0:
        similarity_score = subject_vec.dot(sentence_vec) / (subject_vec_norm * sentence_vec_norm)
    else:
        similarity_score = 0
    
    return similarity_score

In [317]:
# finding subjects from dialogues and narrations
dlg_list = df_lines.index[df_lines['dialogue'].notna()].tolist()

for dlgi in dlg_list:
    if df_lines.loc[(dlgi[0], dlgi[1]), 'dialogue'][-1] == ',':
        subjects = identify_subjects(df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration'])
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k])
    else:
        subjects = identify_subjects(df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration'])
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k])
    df_lines.loc[(dlgi[0], dlgi[1]), 'character'] = relevant_subject

In [318]:
# dialogues and corresponding characters
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        sound                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Deep in the heart of a sprawling city, where s...  
    1                                                  None  
    2      was a haven for those seeking refuge from the...  
1   0     One rainy afternoon, as the sound of raindrops...  
    1                                                  None  
2   0                  Intrigued, Oliver approached Clara.   
    1                                                  None  
3   0                Clara, a twinkle in her eye, replied,   
    1                                                  None  
4   0     Oliver, fueled by curiosity, decided to purcha...

In [319]:
# gathering characters from df_lines
characters = df_lines['character'].value_counts()
df_characters['name'] = characters.index
df_characters['frequency'] = characters.values

In [320]:
# declaring cid as index
df_characters['cid'] = range(0, len(df_characters))
df_characters.set_index('cid', inplace=True)

In [321]:
# characters
df_characters.head(10)

,name,frequency,features
cid,,,
0,clara,2,NaN
1,sound,1,NaN
2,oliver,1,NaN
3,creatures,1,NaN
4,seraphina,1,NaN
5,bookstore,1,NaN
6,that,1,NaN


# 3. Gender Classification

In [322]:
# nlp libraries
import nltk
nltk.download('wordnet')
nltk.download("stopwords")
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [323]:
# defining stopwords set and lemma
set_of_stopwords = set(stopwords.words("english"))
lemma = WordNetLemmatizer()

In [324]:
# function to remove contractions and replace by spaces
def remove_contractions(text):

    contractions = ["ain't", "aren't", "can't", "can't've", "'cause", "could've", "couldn't", "couldn't've", "didn't", "doesn't", "don't",
                    "hadn't", "hadn't've", "hasn't", "haven't", "he'd", "he'd've", "he'll", "he'll've", "he's", "how'd", "how'd'y", "how'll", "how's",
                    "I'd", "I'd've", "I'll", "I'll've", "I'm", "I've", "isn't", "it'd", "it'd've", "it'll", "it'll've", "it's", "let's",
                    "ma'am", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've",
                    "o'clock", "oughtn't", "oughtn't've", "shan't", "sha'n't", "shan't've", "she'd", "she'd've", "she'll", "she'll've", "she's",
                    "should've", "shouldn't", "shouldn't've", "so've", "so's", "that'd", "that'd've", "that's", "there'd", "there'd've", "there's",
                    "they'd", "they'd've", "they'll", "they'll've", "they're", "they've", "to've", "wasn't",
                    "we'd", "we'd've", "we'll", "we'll've", "we're", "we've", "weren't", "what'll", "what'll've", "what're", "what's", "what've",
                    "when's", "when've", "where'd", "where's", "where've", "who'll", "who'll've", "who's", "who've", "why's", "why've",
                    "will've", "won't", "won't've", "would've", "wouldn't", "wouldn't've", "y'all", "y'all'd", "y'all'd've", "y'all're", "y'all've",
                    "you'd", "you'd've", "you'll", "you'll've", "you're", "you've", "gonna"]

    for contraction in contractions:
        text = text.replace(contraction, " ")

    return text

In [325]:
# function to remove punctuations
def remove_punctuations(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [326]:
# function to check noun
def is_noun(word):
    synsets = wordnet.synsets(word)
    for synset in synsets:
        if synset.pos() == 'n':
            return True
    return False

In [327]:
# dialogue to cleaned dialogue
def cleaned_dialogue(text):
    if text is None:
        return ''
    contractionless_text = remove_contractions(text)
    punctuationless_text = remove_punctuations(contractionless_text)
    tokens = nltk.word_tokenize(punctuationless_text)
    filtered_words = [token.lower() for token in tokens if token.lower() not in set_of_stopwords]
    lemmatized_words = [lemma.lemmatize(word) for word in filtered_words]
    words = [word for word in lemmatized_words if is_noun(word)]
    return ' '.join(words)

In [328]:
# list of words for dialogue
dialogue_list = []
for dialogue in df_lines.dialogue:
    dialogue_list.append(cleaned_dialogue(dialogue))
df_lines['cleaned dialogue'] = dialogue_list

In [329]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        sound                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Deep in the heart of a sprawling city, where s...   
    1                                                  None   
    2      was a haven for those seeking refuge from the...   
1   0     One rainy afternoon, as the sound of raindrops...   
    1                                                  None   
2   0                  Intrigued, Oliver approached Clara.    
    1                                                  None   
3   0                Clara, a twinkle in her eye, replied,    
    1                                                  None   
4   0     Oliver, fueled by curiosity, decided to purcha...   

                                           cleaned dialogue  
pid ndid                                                     
0   0                                                        
    1                                                  page  
    2                                                        
1   0                                                        
    1                                           atlas realm  
2   0                                                        
    1                                     story behind book  
3   0                                                        
    1     special one legend hold key realm imagination ...  
4   0

In [330]:
# loading gender count vectorizer
loaded_cv = joblib.load('attributes/gender/gender_cv.joblib')

In [331]:
# creating sparse matrix for transformed values and array
sparse_matrix = loaded_cv.transform(df_lines['cleaned dialogue'])
sparse_matrix_array = sparse_matrix.toarray()

In [332]:
# loading gender model
loaded_model = joblib.load('attributes/gender/gender_model.joblib')

In [333]:
# defining X and predicting
df_lines['gender'] = loaded_model.predict(sparse_matrix_array)

In [334]:
# define gender as None if dialogue is None
df_lines['gender'] = df_lines.apply(lambda row: None if pd.isna(row['dialogue']) else row['gender'], axis=1)

In [335]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        sound                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Deep in the heart of a sprawling city, where s...   
    1                                                  None   
    2      was a haven for those seeking refuge from the...   
1   0     One rainy afternoon, as the sound of raindrops...   
    1                                                  None   
2   0                  Intrigued, Oliver approached Clara.    
    1                                                  None   
3   0                Clara, a twinkle in her eye, replied,    
    1                                                  None   
4   0     Oliver, fueled by curiosity, decided to purcha...   

                                           cleaned dialogue  gender  
pid ndid                                                             
0   0                                                          None  
    1                                                  page  Female  
    2                                                          None  
1   0                                                          None  
    1                                           atlas realm  Female  
2   0                                                          None  
    1                                     story behind book    Male  
3   0                                                          None  
    1     special one legend hold key realm imagination ...  Female  
4   0                                                          None

In [336]:
# defining empty columns
df_characters['male count'], df_characters['female count'] = None, None

In [337]:
# counting male and female values of character
for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'male count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Male")])
    df_characters.loc[df_characters['name'] == character, 'female count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Female")])

In [338]:
# defining gender column
df_characters['gender'] = df_characters.apply(lambda row: 'Female' if row['female count'] > row['male count'] else ('Male' if row['female count'] < row['male count'] else 'None'), axis=1)

In [339]:
# top 10 rows with male and female counts
df_characters.head(10)

,name,frequency,features,male count,female count,gender
cid,,,,,,
0,clara,2,NaN,0,2,Female
1,sound,1,NaN,0,1,Female
2,oliver,1,NaN,1,0,Male
3,creatures,1,NaN,0,1,Female
4,seraphina,1,NaN,1,0,Male
5,bookstore,1,NaN,0,1,Female
6,that,1,NaN,0,1,Female


# 4. Audio Generation

In [169]:
# text-to-speech libraries
from gtts import gTTS
from moviepy.editor import concatenate_audioclips, AudioFileClip
import librosa
import soundfile as sf

In [170]:
# creating audios
print("Step 1: CONVERSIONS")
i = 0
for index, row in df_lines.iterrows():
    if row['dialogue'] is not None:
        speech_gtts = gTTS(text=row['dialogue'], lang='en', slow=False, tld='co.in')
    else:
        speech_gtts = gTTS(text=row['narration'], lang='en', slow=False, tld='ie')
    temp_file = f"conversions/{i}.mp3"
    print(f"Line {i+1}/{df_lines.shape[0]} converted.")
    speech_gtts.save(temp_file)
    i += 1
print("Conversions finished.")

Step 1: CONVERSIONS
Line 1/10 converted.
Line 2/10 converted.
Line 3/10 converted.
Line 4/10 converted.
Line 5/10 converted.
Line 6/10 converted.
Line 7/10 converted.
Line 8/10 converted.
Line 9/10 converted.
Line 10/10 converted.
Conversions finished.


In [120]:
# combining audios
print("Step 2: COMBINATION")
clips = [AudioFileClip(f"conversions/{i}.mp3") for i in range(df_lines.shape[0])]
final_clip = concatenate_audioclips(clips)
print("Combination finished.")

Step 2: COMBINATION
Combination finished.


In [121]:
# adjust speed
print("Step 3: ADJUSTMENT")
final_clip.write_audiofile("conversions/final_slow.mp3")
print("final_slow.mp3 downloaded.")
y, sr = librosa.load("conversions/final_slow.mp3", sr=None)
y_speed = librosa.effects.time_stretch(y, rate=1.25)
print("Speeded over.")

Step 3: ADJUSTMENT
MoviePy - Writing audio in conversions/final_slow.mp3


MoviePy - Done.
final_slow.mp3 downloaded.
Speeded over.


In [122]:
# removing audios
print("Step 4: REMOVAL")
for i in range(df_lines.shape[0]):
    os.remove(f"conversions/{i}.mp3")
    print(f"Removed {i}.mp3")
os.remove("conversions/final_slow.mp3")
print("Removal over.")

Step 4: REMOVAL
Removed 0.mp3
Removed 1.mp3
Removed 2.mp3
Removed 3.mp3
Removed 4.mp3
Removed 5.mp3
Removed 6.mp3
Removed 7.mp3
Removed 8.mp3
Removed 9.mp3
Removed 10.mp3
Removed 11.mp3
Removed 12.mp3
Removed 13.mp3
Removed 14.mp3
Removed 15.mp3
Removal over.


In [123]:
# downloading final audio
print("Step 5: DOWNLOADING")
final_title = title.replace(" ", "_")
sf.write(f"audiobooks/{final_title}.mp3", y_speed, sr)
print(f"{final_title}.mp3 downloaded")

Step 5: DOWNLOADING
beautiful_garden.mp3 downloaded
